# Product Review Sentiment Analysis + Reasoning

An automated tool to classify customer reviews and understand emotional drivers behind feedback for e-commerce teams.

## Features:
- Sentiment classification (Positive/Neutral/Negative)
- LLM-generated explanation: "Why is this review negative?"
- Option to rephrase in a neutral or brand-friendly tone
- Export capability to internal ticketing system

---

## 1. Setup and Installation

In [12]:
# Install required packages
!pip install transformers torch gradio openai textblob pyperclip

In [13]:
import gradio as gr
import openai
import json
import pandas as pd
from datetime import datetime
from textblob import TextBlob
from transformers import pipeline
import os
import pyperclip

## 2. Configuration

In [14]:
try:
    from google.colab import userdata
    openai.api_key = userdata.get('OPENAI_API_KEY')
except:
    # Fallback to environment variable
    openai.api_key = os.getenv('OPENAI_API_KEY')

if not openai.api_key:
    print("⚠️ OpenAI API key not found. Please set it using one of the methods above.")
else:
    print("✅ OpenAI API key configured successfully!")

✅ OpenAI API key configured successfully!


## 3. Core Functions

In [15]:
# Initialize sentiment analysis pipeline
try:
    # Use a lightweight model for faster inference
    sentiment_pipeline = pipeline("sentiment-analysis",
                                model="cardiffnlp/twitter-roberta-base-sentiment-latest",
                                return_all_scores=True)
    print("✅ Sentiment analysis model loaded successfully!")
except Exception as e:
    print(f"⚠️ Error loading sentiment model: {e}")
    print("Falling back to TextBlob for sentiment analysis...")
    sentiment_pipeline = None

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


✅ Sentiment analysis model loaded successfully!


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [16]:
def analyze_sentiment(text):
    """
    Analyze sentiment of the given text
    Returns: (sentiment_label, confidence_score)
    """
    if not text.strip():
        return "Neutral", 0.0

    try:
        if sentiment_pipeline:
            # Use transformer model
            results = sentiment_pipeline(text)[0]

            # Map labels to our format
            label_mapping = {
                'LABEL_0': 'Negative',
                'LABEL_1': 'Neutral',
                'LABEL_2': 'Positive'
            }

            best_result = max(results, key=lambda x: x['score'])
            sentiment = label_mapping.get(best_result['label'], best_result['label'])
            confidence = best_result['score']

        else:
            # Fallback to TextBlob
            blob = TextBlob(text)
            polarity = blob.sentiment.polarity

            if polarity > 0.1:
                sentiment = "Positive"
            elif polarity < -0.1:
                sentiment = "Negative"
            else:
                sentiment = "Neutral"

            confidence = abs(polarity)

    except Exception as e:
        print(f"Error in sentiment analysis: {e}")
        return "Neutral", 0.0

    return sentiment, confidence

In [17]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# Your existing functions remain the same
def get_sentiment_explanation(review_text, sentiment):
    """
    Generate explanation for why the review has the given sentiment
    """
    try:
        prompt = f"""
        Analyze this customer review and explain why it has a {sentiment.lower()} sentiment.

        Review: "{review_text}"

        Provide a clear, concise explanation focusing on:
        - Key emotional indicators
        - Specific issues or praise mentioned
        - Language tone and intensity

        Keep the explanation under 100 words and professional.
        """

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in sentiment analysis and customer feedback interpretation."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.3
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"Error generating explanation: {str(e)}"


def rephrase_review(review_text, tone="neutral"):
    """
    Rephrase the review in a neutral or brand-friendly tone
    """
    try:
        if tone == "neutral":
            tone_instruction = "neutral, objective tone while preserving the core feedback"
        else:  # brand-friendly
            tone_instruction = "constructive, brand-friendly tone that maintains the feedback but reduces negativity"

        prompt = f"""
        Rephrase this customer review in a {tone_instruction}.

        Original review: "{review_text}"

        Guidelines:
        - Maintain the core message and specific issues mentioned
        - Use professional, respectful language
        - Keep the same length approximately
        - Focus on constructive feedback rather than emotional language

        Rephrased review:
        """

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in professional communication and customer service."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            temperature=0.4
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"Error rephrasing review: {str(e)}"

## 4. Export Functions

In [18]:
# Global variable to store analysis results
analysis_results = []

def export_to_json(results):
    """
    Export results to JSON format for ticketing system
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"sentiment_analysis_{timestamp}.json"

    export_data = {
        "export_timestamp": datetime.now().isoformat(),
        "total_reviews": len(results),
        "reviews": results
    }

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)

    return filename

def copy_to_clipboard(text):
    """
    Copy text to clipboard
    """
    try:
        pyperclip.copy(text)
        return "✅ Copied to clipboard!"
    except Exception as e:
        return f"❌ Error copying to clipboard: {str(e)}"

## 5. User Interface

In [19]:
def analyze_review(review_text):
    """
    Main function to analyze a review
    """
    if not review_text.strip():
        return "Please enter a review to analyze.", "", "", ""

    # Analyze sentiment
    sentiment, confidence = analyze_sentiment(review_text)

    # Create sentiment badge
    confidence_pct = f"{confidence:.1%}"

    if sentiment == "Positive":
        badge_color = "🟢"
    elif sentiment == "Negative":
        badge_color = "🔴"
    else:
        badge_color = "🟡"

    sentiment_display = f"{badge_color} {sentiment} ({confidence_pct})"

    # Generate explanation
    explanation = get_sentiment_explanation(review_text, sentiment)

    # Store result
    result = {
        "timestamp": datetime.now().isoformat(),
        "review_text": review_text,
        "sentiment": sentiment,
        "confidence": confidence,
        "explanation": explanation
    }

    analysis_results.append(result)

    return sentiment_display, explanation, "", ""

def rephrase_current_review(review_text, tone):
    """
    Rephrase the current review
    """
    if not review_text.strip():
        return "Please enter a review first."

    rephrased = rephrase_review(review_text, tone)
    return rephrased

def export_results():
    """
    Export all analysis results
    """
    if not analysis_results:
        return "No analysis results to export."

    filename = export_to_json(analysis_results)
    return f"✅ Results exported to {filename}"

def copy_analysis_result(sentiment_display, explanation, rephrased_text):
    """
    Copy analysis result to clipboard
    """
    result_text = f"Sentiment: {sentiment_display}\n\nExplanation: {explanation}"
    if rephrased_text.strip():
        result_text += f"\n\nRephrased: {rephrased_text}"

    return copy_to_clipboard(result_text)

In [20]:
# Create Gradio interface
def create_interface():
    with gr.Blocks(title="Product Review Sentiment Analysis", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 🔍 Product Review Sentiment Analysis + Reasoning

            Analyze customer reviews, understand emotional drivers, and rephrase feedback for your e-commerce team.
            """
        )

        with gr.Row():
            with gr.Column(scale=2):
                # Input section
                review_input = gr.Textbox(
                    label="Customer Review",
                    placeholder="Enter the customer review here...",
                    lines=4,
                    max_lines=8
                )

                analyze_btn = gr.Button("🔍 Analyze Review", variant="primary", size="lg")

            with gr.Column(scale=2):
                # Results section
                sentiment_output = gr.Textbox(
                    label="Sentiment Analysis",
                    interactive=False
                )

                explanation_output = gr.Textbox(
                    label="Explanation",
                    lines=4,
                    interactive=False
                )

        # Rephrasing section
        gr.Markdown("### 📝 Rephrase Review")

        with gr.Row():
            tone_radio = gr.Radio(
                choices=["neutral", "brand-friendly"],
                value="neutral",
                label="Rephrasing Tone"
            )
            rephrase_btn = gr.Button("📝 Rephrase", variant="secondary")

        rephrased_output = gr.Textbox(
            label="Rephrased Review",
            lines=4,
            interactive=True,
            placeholder="Rephrased version will appear here..."
        )

        # Export section
        gr.Markdown("### 📤 Export & Copy")

        with gr.Row():
            copy_btn = gr.Button("📋 Copy to Clipboard", variant="secondary")
            export_btn = gr.Button("💾 Export All Results", variant="secondary")

        status_output = gr.Textbox(
            label="Status",
            interactive=False,
            show_label=False
        )

        # Event handlers
        analyze_btn.click(
            fn=analyze_review,
            inputs=[review_input],
            outputs=[sentiment_output, explanation_output, rephrased_output, status_output]
        )

        rephrase_btn.click(
            fn=rephrase_current_review,
            inputs=[review_input, tone_radio],
            outputs=[rephrased_output]
        )

        copy_btn.click(
            fn=copy_analysis_result,
            inputs=[sentiment_output, explanation_output, rephrased_output],
            outputs=[status_output]
        )

        export_btn.click(
            fn=export_results,
            inputs=[],
            outputs=[status_output]
        )

        # Example reviews
        gr.Markdown(
            """
            ### 📋 Example Reviews to Try:

            **Negative:** "The product broke after two weeks. Very disappointed with the quality. Would not recommend."

            **Positive:** "Amazing product! Works exactly as described and arrived quickly. Highly recommend!"

            **Neutral:** "The app is hard to navigate. Some features work well, others could be improved."
            """
        )

    return demo

## 6. Launch the Application

In [24]:
# Create and launch the interface
demo = create_interface()

# Launch with public sharing enabled for Colab
demo.launch(
    share=True,  # Creates a public link
    debug=True,  # Enable debug mode
    show_error=True  # Show errors in the interface
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://80d06c15619442d292.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://80d06c15619442d292.gradio.live


## 📖 Usage Notes

### Setup:
1. **OpenAI API Key**: Set your OpenAI API key using Google Colab secrets or environment variables
2. **Run all cells**: Execute all cells in order to set up the application
3. **Launch**: The final cell will create a public Gradio link you can share

### Features:
- **Sentiment Analysis**: Automatically classifies reviews as Positive, Negative, or Neutral
- **AI Explanations**: Uses GPT to explain why a review has a particular sentiment
- **Review Rephrasing**: Converts negative reviews to neutral or brand-friendly tone
- **Export Options**: Copy results to clipboard or export all analyses to JSON

### For Production Use:
- Consider using a more robust database for storing results
- Implement user authentication and access controls
- Add rate limiting for API calls
- Set up proper logging and monitoring

### Troubleshooting:
- If sentiment analysis fails, the app falls back to TextBlob
- Ensure your OpenAI API key has sufficient credits
- Check the status output for any error messages
